- **Created by Zehao Li (Takuho Ri)**
- **Created on 2025-02-14 (Fri)  14:57:03 (+09:00)**

transform all molecule from smiles to graph (for AttentiveFP)

In [1]:
import os
import sys
from tqdm.notebook import tqdm
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import Data

project_path = os.getcwd().split("/prep")[0]
sys.path.append(project_path)

In [2]:
from src.AttentiveFP.featurizer import _prep_feats

In [3]:
csv_dir = os.path.join(project_path, "data", "tox_csv")
graph_dir = os.path.join(project_path, "data", "AttentiveFP_graphs")

os.makedirs(graph_dir, exist_ok=True)

### prepare graphs except BACE

In [5]:
for data in os.listdir(csv_dir):
    if data == "bace.csv":
        continue
    print(f"=== {data.replace(".csv", "")} ===")
    df = pd.read_csv(os.path.join(csv_dir, data), index_col=0)
    smiles = np.array(df["cano_smi"])
    ys = np.array(df[df.columns.values[2:]])
    assert len(smiles) == len(ys), "difficult length between smiles and y !"
    graphs = []
    for smi, y in tqdm(zip(smiles, ys), total=len(smiles)):
        y = torch.tensor(y).float().view(1, -1)
        atom_feats, edge_idx, edge_feats = _prep_feats(smi)
        graph = Data(atom_feats, edge_idx, edge_feats, y)
        graphs.append(graph)
    with open(os.path.join(graph_dir, f"{data.replace(".csv", ".pkl")}"), "wb") as f:
        pickle.dump(graphs, f)

=== sider ===


  0%|          | 0/1384 [00:00<?, ?it/s]

[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:50] WARNING: not removing hydrogen atom without neighbors
[15:35:52] WARNING: not removing hydrogen atom without neighbors
[15:35:52] WARNING: not removing hydrogen atom without neighbors
[15:35:52] WARNING: not removing hydrogen atom without neighbors
[15:35:53] WARNING: not removing hydrogen atom without neighbors
[15:35:53] WARNING: not removing hydrogen atom without neighbors


=== herg_karim ===


  0%|          | 0/13445 [00:00<?, ?it/s]

...

[15:37:51] WARNING: not removing hydrogen atom without neighbors


=== clintox_M ===


  0%|          | 0/1468 [00:00<?, ?it/s]

=== ld50 ===


  0%|          | 0/7385 [00:00<?, ?it/s]

### prepare graphs for BACE

In [7]:
print("=== bace ===")
df = pd.read_csv(os.path.join(csv_dir, "bace.csv"), index_col=0)
smiles = np.array(df["cano_smi"])
ys_c = np.array(df[["Class"]])
ys_r = np.array(df[["pIC50"]])
assert len(smiles) == len(ys_c) == len(ys_r), "difficult length between smiles and y !"
graphs_c = []
graphs_r = []
cnt = 0
for smi, y_c, y_r in tqdm(zip(smiles, ys_c, ys_r), total=len(smiles)):
    y_c = torch.tensor(y_c).float().view(1, -1)
    y_r = torch.tensor(y_r).float().view(1, -1)
    atom_feats, edge_idx, edge_feats = _prep_feats(smi)
    graph_c = Data(atom_feats, edge_idx, edge_feats, y_c)
    graph_r = Data(atom_feats, edge_idx, edge_feats, y_r)
    graphs_c.append(graph_c)
    graphs_r.append(graph_r)
    if cnt == 0:
        print(y_c, y_c.shape)
        print(y_r, y_r.shape)
        cnt += 1
with open(os.path.join(graph_dir, f"bace_c.pkl"), "wb") as f:
    pickle.dump(graphs_c, f)
with open(os.path.join(graph_dir, f"bace_r.pkl"), "wb") as f:
    pickle.dump(graphs_r, f)

=== bace ===


  0%|          | 0/1513 [00:00<?, ?it/s]

tensor([[1.]]) torch.Size([1, 1])
tensor([[9.1549]]) torch.Size([1, 1])
